In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
cd ~/demres

/Users/zurfarosa/demres


In [3]:
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.tools.tools import add_constant
import pylab as pl
from datetime import date, timedelta

import demres
from demres.common.constants import entry_type
from demres.demins.constants import Study_Design as sd
from demres.common import codelists
from demres.common.helper_functions import *
from demres.demins.statistical_functions import *

In [4]:
pd.set_option('display.max_columns', None)

## Specifiy dementia subtype

In [5]:
subtype = 'all_dementia' # options: 'alzheimers', 'vascular','all_dementia'

## Specify exposure window

In [6]:
window = '12_to_7' #options: '12_to_7','10_to_5','8_to_3'

## Load relevant dataframe and create intercept

In [7]:
pt_features = pd.read_csv('data/pt_data/processed_data/pt_features_demins_' + subtype + '_' + window +'.csv',delimiter=',',parse_dates=['index_date','data_end','data_start'],infer_datetime_format=True)

In [8]:
pt_features.sample(5)

,patid,yob,pracid,female,index_date,isCase,final dementia medcode,data_end,earliest_sysdate,sysdate_of_final_converted_code,start_of_year_after_earliest_year_with_>15_consultations,data_start,matchid,age_at_index_date
28590,1990258,31,258,0,2011-05-23,True,1917.0,2013-04-22,2001-11-12,NaN,1992-01-01,1992-01-01,39857,80
5810,19452287,36,287,1,2010-12-31,True,NaN,2013-04-09,2002-01-21,NaN,1998-01-01,1998-01-01,43866,74
28548,11672178,22,178,1,2009-06-30,True,7664.0,2013-04-18,1998-12-14,NaN,1992-01-01,1992-01-01,26868,87
1341,265598,36,598,0,2008-01-15,False,NaN,2010-01-26,1999-12-09,NaN,1996-01-01,1996-01-01,16719,72
27683,12393245,37,245,0,2009-12-31,True,11379.0,2012-08-30,2002-03-14,NaN,1992-01-01,1992-01-01,37463,72


In [9]:
pt_features.columns

Index(['patid', 'yob', 'pracid', 'female', 'index_date', 'isCase',
       'final dementia medcode', 'data_end', 'earliest_sysdate',
       'sysdate_of_final_converted_code',
       'start_of_year_after_earliest_year_with_>15_consultations',
       'data_start', 'matchid', 'age_at_index_date'],
      dtype='object')

In [10]:
pt_features['intercept'] = 1.0

In [11]:
# pt_features.sort_values(by='insomnia_count<=10',ascending=False)

In [12]:
columns_always_for_inclusion = [
    'female',
    'non_insomnia_GP_consultations', 'stroke',
    'non_stroke_vascular_disease', 'hypertension',
    'diabetes', 'mental_illness_non_smi', 'mental_illness_smi',
    'sleep_apnoea', 'chronic_pulmonary_disease', 'epilepsy',
    'mood_stabilisers_100_pdds',
    'other_sedatives_100_pdds', 'antidepressants_100_pdds', 
    'depot_antipsychotics_100_pdds', 'antipsychotics_100_pdds',
    'benzo_and_z_drugs_100_pdds'
]

## Insomnia consultations

### insomnia yes/no

In [13]:
cols_for_inclusion_here = ['insomnia_any']
training_cols = [col for col in pt_features.columns if col in cols_for_inclusion_here+columns_always_for_inclusion]
summary_table,univariate_and_multivariate_results = purposefully_select_covariates(pt_features,covariates=training_cols,main_variables=cols_for_inclusion_here)

female  being retained as mean > 0


*Stage 1*
Univariate results
        Univariate OR  p value  [0.025  0.975]
female            1.0      1.0   0.973   1.028

The following covariates were retained during stage 1:

[]

The following covariates were NOT retained during stage 1, as p value > 0.25:

['female']


LinAlgError: Arrays cannot be empty

In [ ]:
univariate_and_multivariate_results

### insomnia (continuous variable)

In [ ]:
cols_for_inclusion_here = ['insomnia','benzo_and_z_drugs_100_pdds']
training_cols = [col for col in pt_features.columns if col in cols_for_inclusion_here+columns_always_for_inclusion]
summary_table,univariate_and_multivariate_results = purposefully_select_covariates(pt_features,covariates=training_cols,main_variables=cols_for_inclusion_here)

In [ ]:
univariate_and_multivariate_results

### insomnia quantiles

In [ ]:
cols_for_inclusion_here = ['insomnia_count:1_5','insomnia_count:above_5','benzo_and_z_drugs_100_pdds']
training_cols = [col for col in pt_features.columns if col in cols_for_inclusion_here+columns_always_for_inclusion]
summary_table,univariate_and_multivariate_results = purposefully_select_covariates(pt_features,covariates=training_cols,main_variables=cols_for_inclusion_here)

In [ ]:
univariate_and_multivariate_results